## Connecting to db

In [2]:
import sqlite3
import pandas as pd

In [3]:
!ls

Basics.ipynb  nominations.db


In [4]:
conn = sqlite3.connect('nominations.db')

## Quering table in db

In [10]:
conn.execute("DROP TABLE ceremonies;")

In [11]:
q1 = "PRAGMA table_info(nominations);"
schema = conn.execute(q1).fetchall()
for i in schema:
    print(i)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [12]:
q2 = "SELECT * FROM nominations LIMIT 10;"
first_ten = conn.execute(q2).fetchall()
for i in first_ten:
    print(i)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


In [13]:
q3 = "SELECT * FROM sqlite_master WHERE type='table';"
conn.execute(q3).fetchall()

[('table',
  'nominations',
  'nominations',
  2,
  'CREATE TABLE "nominations" (\n"Year" INTEGER,\n  "Category" TEXT,\n  "Nominee" TEXT,\n  "Won" INTEGER,\n  "Movie" TEXT,\n  "Character" TEXT\n)')]

In [14]:
pd.read_sql(q3, conn)

,type,name,tbl_name,rootpage,sql
0,table,nominations,nominations,2,"CREATE TABLE ""nominations"" (\n""Year"" INTEGER,\..."


## Creating a new table in db

In [15]:
q4 = "CREATE TABLE ceremonies (id INTEGER PRIMARY KEY, Year INTEGER, Host TEXT);"
conn.execute(q4)

In [16]:
# let's check created table now
pd.read_sql(q3, conn)

,type,name,tbl_name,rootpage,sql
0,table,nominations,nominations,2,"CREATE TABLE ""nominations"" (\n""Year"" INTEGER,\..."
1,table,ceremonies,ceremonies,19,CREATE TABLE ceremonies (id INTEGER PRIMARY KE...


In [17]:
pd.read_sql("PRAGMA table_info(ceremonies)", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,Year,INTEGER,0,None,0
2,2,Host,TEXT,0,None,0


## Filling table with values

In [18]:
# First we create a list of tuples with values
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal")
              ]
years_hosts

[(2010, 'Steve Martin'),
 (2009, 'Hugh Jackman'),
 (2008, 'Jon Stewart'),
 (2007, 'Ellen DeGeneres'),
 (2006, 'Jon Stewart'),
 (2005, 'Chris Rock'),
 (2004, 'Billy Crystal'),
 (2003, 'Steve Martin'),
 (2002, 'Whoopi Goldberg'),
 (2001, 'Steve Martin'),
 (2000, 'Billy Crystal')]

In [19]:
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?, ?);"
conn.executemany(insert_query, years_hosts)

In [20]:
# checking table after insert
first_ten = "SELECT * FROM ceremonies;"
pd.read_sql(first_ten, conn)

,id,Year,Host
0,1,2010,Steve Martin
1,2,2009,Hugh Jackman
2,3,2008,Jon Stewart
3,4,2007,Ellen DeGeneres
4,5,2006,Jon Stewart
5,6,2005,Chris Rock
6,7,2004,Billy Crystal
7,8,2003,Steve Martin
8,9,2002,Whoopi Goldberg
9,10,2001,Steve Martin


## Creating relations using foreign keys (one-to-many)

In [21]:
# First we need to turn on foreign keys check 
# (Whenever we try inserting a row into a table containing foreign key(s), 
# SQLite will query the linked table to make sure that foreign key value exists)
conn.execute("PRAGMA foreign_keys = ON;")

# this needs to be run every time we connect to a database
# where we'll be inserting foreign keys. 

We want now to replace `nominations.year` with a column `ceremony_id` that will be inked to `ceremonies.id`.

As in sqlite we can't delete a column in a table we'll have to work around.

In [26]:
# first we create a table `nominations_two` with a foreign key to `ceremonies.id`

create_table = '''CREATE TABLE nominations_two (
id INTEGER PRIMARY KEY,
category TEXT,
nominee TEXT,
movie TEXT,
character TEXT,
won INTEGER,
ceremony_id INTEGER,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id))'''
conn.execute(create_table)

In [27]:
# checking result 
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,nominations,nominations,2,"CREATE TABLE ""nominations"" (\n""Year"" INTEGER,\..."
1,table,ceremonies,ceremonies,19,CREATE TABLE ceremonies (id INTEGER PRIMARY KE...
2,table,nominations_two,nominations_two,20,CREATE TABLE nominations_two (\nid INTEGER PRI...


In [28]:
# then we fetch the needed data from `nominations` + `ceremonies` (with inner join)
# [but without any foreign keys here btw...]

join_query = '''SELECT 
nominations.category, 
nominations.nominee, 
nominations.movie, 
nominations.character, 
nominations.won, 
ceremonies.id
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;'''

# we store it in the list of tuples (to use with `.executemany` after)

joined_nominations = conn.execute(join_query).fetchall()
joined_nominations[:3]

[('Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 ('Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 ('Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1)]

In [29]:
# now we create an insert query with placeholders

insert_nominations_two = '''INSERT INTO nominations_two (
category, nominee, movie, character, won, ceremony_id)
VALUES (?, ?, ?, ?, ?, ?)'''

In [30]:
# inserting to `nominations_two`

conn.executemany(insert_nominations_two, joined_nominations)

In [31]:
# checking result

pd.read_sql("SELECT * FROM nominations_two LIMIT 5;", conn)

,id,category,nominee,movie,character,won,ceremony_id
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,2,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,3,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,4,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,5,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1


In [32]:
# now we can delete initial `nominations` table
conn.execute("DROP TABLE nominations;")

In [33]:
# check that it's done
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,ceremonies,ceremonies,19,CREATE TABLE ceremonies (id INTEGER PRIMARY KE...
1,table,nominations_two,nominations_two,20,CREATE TABLE nominations_two (\nid INTEGER PRI...


In [34]:
# and rename new table
conn.execute("ALTER TABLE nominations_two RENAME TO nominations;")

In [35]:
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,ceremonies,ceremonies,19,CREATE TABLE ceremonies (id INTEGER PRIMARY KE...
1,table,nominations,nominations,20,"CREATE TABLE ""nominations"" (\nid INTEGER PRIMA..."


## Creating join table (many-to-many)

In [36]:
create_movies = '''CREATE TABLE movies (
id INTEGER PRIMARY KEY, 
movie TEXT);'''

create_actors = '''CREATE TABLE actors (
id INTEGER PRIMARY KEY, 
actor TEXT);'''

create_movies_actors = '''CREATE TABLE movies_actors (
id INTEGER PRIMARY KEY,
movie_id INTEGER REFERENCES movies(id),
actor_id INTEGER REFERENCES actors(id));'''

conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)

In [37]:
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,ceremonies,ceremonies,19,CREATE TABLE ceremonies (id INTEGER PRIMARY KE...
1,table,nominations,nominations,20,"CREATE TABLE ""nominations"" (\nid INTEGER PRIMA..."
2,table,movies,movies,2,"CREATE TABLE movies (\nid INTEGER PRIMARY KEY,..."
3,table,actors,actors,4,"CREATE TABLE actors (\nid INTEGER PRIMARY KEY,..."
4,table,movies_actors,movies_actors,5,CREATE TABLE movies_actors (\nid INTEGER PRIMA...


### Populating the movies and actors tables

In [ ]:
insert_movies = "INSERT INTO movies (movie) SELECT DISTINCT movie FROM nominations;"
insert_actors = "INSERT INTO actors (actor) SELECT DISTINCT nominee FROM nominations;"

conn.execute(insert_movies)
conn.execute(insert_actors)

In [40]:
print(pd.read_sql("select * from movies limit 5;", conn))
print("---")
print(pd.read_sql("select * from actors limit 5;", conn))

   id               movie
0   1            Biutiful
1   2           True Grit
2   3  The Social Network
3   4   The King's Speech
4   5           127 Hours
---
   id            actor
0   1    Javier Bardem
1   2     Jeff Bridges
2   3  Jesse Eisenberg
3   4      Colin Firth
4   5     James Franco


### Populating a join table

In [41]:
pairs_query = "SELECT movie, nominee FROM nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()
movie_actor_pairs[:5]

[('Biutiful', 'Javier Bardem'),
 ('True Grit', 'Jeff Bridges'),
 ('The Social Network', 'Jesse Eisenberg'),
 ("The King's Speech", 'Colin Firth'),
 ('127 Hours', 'James Franco')]

In [42]:
join_table_insert = '''INSERT INTO movies_actors (movie_id, actor_id)
VALUES (
(SELECT id FROM movies WHERE movie == ?),
(SELECT id FROM actors WHERE actor == ?));'''

conn.executemany(join_table_insert,movie_actor_pairs)

In [43]:
pd.read_sql("SELECT * FROM movies_actors LIMIT 10;", conn)

,id,movie_id,actor_id
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4
4,5,5,5
5,6,6,6
6,7,7,7
7,8,8,8
8,9,9,9
9,10,4,10


## The end

In [44]:
conn.close()